
___
# Conducting a Parameter Estimation
___

Author: Chenyu Wang

For this demonstration, we will be going through how to set up a parameter estimation using the Pyomo tool ***parmest***. This simple case study aims to validate the reverse osmosis model by the real plant data.

### Step 1: Imports
Import all Pyomo, WaterTAP and helper modules needed for building and solving the parameter estimation problem.

In [122]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    TerminationCondition,
    value,
    Reals,
    TransformationFactory,
    assert_optimal_termination,
    units as pyunits,
)
from pyomo.network import Arc
import pyomo.contrib.parmest.parmest as parmest  # to perform the parameter estimation

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed, Separator
from idaes.core.util.initialization import propagate_state
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.core.util.scaling as iscale
import idaes.logger as idaeslog

# Imports from WaterTAP
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D as RO,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
)
from watertap.property_models import seawater_prop_pack as props
from watertap.core.solvers import get_solver  # to bring in ipopt solver

# Other imports
import pandas as pd  # to create a pandas dataframe to organize the data

pd.options.mode.chained_assignment = None
import numpy as np  # to manipulate the data into a usable format
import matplotlib.pyplot as plt  # to plot the results
# from utility_functions import load_data
import logging

logging.getLogger("pyomo").setLevel(logging.CRITICAL)
import warnings

warnings.filterwarnings("ignore")

### Step 2: Gather and prepare the data
*How does data need to be formatted for parmest?*


- **Pandas Dataframe:** each column is an observed quantity (temperature, concentration, vapor pressure, etc.), each row is a distinct scenario (25, 0.02, 31.33)

**Other options:**
- **List of dictionaries:** each entry of the list is a distinct scenario, each key an observed quantity 
- **List of json file names:** each entry of the list contains a json file with the distinct scenario (for large datasets in parallel computing)

In this tutorial, we use the data collected from a pilot plan of **Orange County Water District (OCWD)** for a RO unit.

In [123]:
# Read in xlsx file to pd.dataframe
data = pd.read_csv("WRD_Data_PRO1.csv")
# raw_data = pd.read_excel(r"Plant_data.xlsx")
# Prepare the data
# data, full_data = load_data(raw_data)
display(data)

,DateTime,stage 1 feed conductivity (us/cm),stage 1 feed pressure (psi),stage 1 permeate flowrate (gpm),stage 1 permeate conductivity (us/cm),stage 1 concentrate flowrate (gpm),stage 1 concentrate pressure (psi),Unnamed: 7,stage2_ro_inlet_P (psi),stage2_ro_outlet_P (psi),stage2_feed_conductivity (us/cm),stage2_perm_conductivity (us/cm),stage2_feed_flow (gpm),stage2_perm_flow (gpm),stage3_ro_inlet_P (psi),stage3_ro_outlet_P (psi),stage3_feed_conductivity (us/cm),stage3_perm_conductivity (us/cm),stage3_feed_flow (gpm),stage3_perm_flow (gpm)
0,2/7/2019,1106.280029,25.14999962,763.1300049,0,615.1599731,24.76999855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/27/2019,1314.809937,67.94999695,1437.959961,0,955.1999817,66.26999664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2/28/2019,1263.189941,105.409996,1760.179932,0,1125.509979,102.7699966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/7/2019,1026.199951,70,1066.599976,24,878.3799744,62.44999695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3/9/2019,1057,97.18000031,1600.140015,26,1225.159973,84.15000153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,7/27/2024,1112.549927,124.2699966,1817.689941,39.02999878,801.3999634,115.2599945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1708,7/28/2024,1076.48999,127.0499954,1876.469971,38,801.6599731,118.1800003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1709,7/29/2024,1048.73999,126.6999969,1842.640015,37.02999878,783.4499817,117.6899948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1710,7/30/2024,1070.109985,13.50999928,0.439999998,137.9700012,0.359999981,13.65999985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add and Values that need to be calculated

In [124]:
# Convert all columns except DateTime to numeric
for col in data.columns:
    if col != "DateTime":
        data[col] = pd.to_numeric(data[col], errors='coerce')

# Do any required data cleaning
data_mask_date = (data["DateTime"].str.startswith("8/1")) & (data["DateTime"].str.contains("/2021"))
cleaned_data = data[data_mask_date].reset_index(drop=True)
data_mask_perm_flow = cleaned_data["stage 1 permeate flowrate (gpm)"] >= 1603
cleaned_data = cleaned_data[data_mask_perm_flow].reset_index(drop=True)

cleaned_data["stage 1 feed salinity (g/L)"] = cleaned_data["stage 1 feed conductivity (us/cm)"] * 0.0005 # Conversion factor
cleaned_data["stage 1 permeate salinity (g/L)"] = cleaned_data["stage 1 permeate conductivity (us/cm)"] * 0.0005
cleaned_data["stage 1 feed flowrate (gpm)"] = cleaned_data["stage 1 permeate flowrate (gpm)"] + cleaned_data["stage 1 concentrate flowrate (gpm)"]
display(cleaned_data)

,DateTime,stage 1 feed conductivity (us/cm),stage 1 feed pressure (psi),stage 1 permeate flowrate (gpm),stage 1 permeate conductivity (us/cm),stage 1 concentrate flowrate (gpm),stage 1 concentrate pressure (psi),Unnamed: 7,stage2_ro_inlet_P (psi),stage2_ro_outlet_P (psi),...,stage2_perm_flow (gpm),stage3_ro_inlet_P (psi),stage3_ro_outlet_P (psi),stage3_feed_conductivity (us/cm),stage3_perm_conductivity (us/cm),stage3_feed_flow (gpm),stage3_perm_flow (gpm),stage 1 feed salinity (g/L),stage 1 permeate salinity (g/L),stage 1 feed flowrate (gpm)
0,8/11/2021,1077.169922,144.389999,1608.400024,25.000000,1049.279938,131.699997,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.538585,0.012500,2657.679962
1,8/13/2021,1094.020020,143.289993,1637.250000,25.969999,1043.739990,131.020004,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.547010,0.012985,2680.989990
2,8/16/2021,962.019959,141.110001,1607.659912,24.000000,1039.359985,128.789993,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.481010,0.012000,2647.019897
3,8/17/2021,1064.669922,143.959992,1604.239990,26.000000,1038.019989,131.959992,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.532335,0.013000,2642.259979
4,8/18/2021,1075.719971,142.020004,1633.709961,26.000000,1029.359955,130.199997,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.537860,0.013000,2663.069916


### Step 3: Define the RO model for parameter estimation
*What model are parameters being fit to?*

***Parmest*** requires a "model function" to be defined that takes in the data and returns a Pyomo model.

Set up the Pyomo model defining:
- Pyomo Vars or Params for each parameter (or 'theta') to be estimated
- the model equation (a function of the observed data, i.e., permeate flow rate, mass fraction)

 

For this example, the RO model we are proposing is defined as `ro_parmest` function:


In [125]:
# Set the solver
solver = get_solver()  # this will make the ipopt solver available

# Define global conversion factors
# psi_to_pascal = 6894.75  # Pressure conversion
# gpm_to_m3ps = 6.309e-005  # Vol. flow conversion

In [126]:
# Import the fuction that creates flowsheet
from wrd.membrane_properties.ro_memb_test import solve_ro_module
def ro_parmest(data):
    # Maybe add logic to check units
    Qin = data.iloc[0]["stage 1 feed flowrate (gpm)"]
    Cin = data.iloc[0]["stage 1 feed salinity (g/L)"]
    Pin = data.iloc[0]["stage 1 feed pressure (psi)"]
    Pout = data.iloc[0]["stage 1 concentrate pressure (psi)"]
    # print(Qin, Cin, Pin, Pout)
    m = solve_ro_module(Qin=Qin, Cin=Cin, Pin=Pin, Pout=Pout)

    return m


### Step 4: Define a list of parameter names

The variables to be estimated by parmest must be given as a list of strings of the variable names as they are defined in the `ro_parmest`. 

In [127]:
# variables from model to be estimated
# required format: list with strings of param/var names
theta_names = ["fs.ro.unit.A_comp[0, 'H2O']", "fs.ro.unit.B_comp[0, 'NaCl']"]

### Step 5: Define an objective function

Now, we should define an objective function for the parameter estimation. This is the deviation between the observation and the prediction typically chosen to be the sum of squared errors.

$$
\sum_{i=0}^n (observation_i - prediction_i)^2 
$$


In [128]:
# Sum of Squared Errors function
def SSE(m, data):
    flow_vol_scale = 1 / 1e3  
    conc_mass_phase_comp_scale = 1 / 1e-2 # Scaling factors are inverse of magnitude of input value???
    expr = flow_vol_scale * (
        (
            float(data.iloc[0]["stage 1 permeate flowrate (gpm)"])
            - value(pyunits.convert(m.fs.ro.unit.mixed_permeate[0.0].flow_vol_phase["Liq"], pyunits.gal/pyunits.min))
        )
        ** 2
    ) + conc_mass_phase_comp_scale * (
        (
            float(data.iloc[0]["stage 1 permeate salinity (g/L)"])
            - value(pyunits.convert(m.fs.ro.unit.mixed_permeate[0.0].conc_mass_phase_comp["Liq", "NaCl"], pyunits.g / pyunits.L))
        )
        ** 2
    )
    print(float(data.iloc[0]["stage 1 permeate salinity (g/L)"])
            - value(pyunits.convert(m.fs.ro.unit.mixed_permeate[0.0].conc_mass_phase_comp["Liq", "NaCl"], pyunits.g / pyunits.L)))
    print(float(data.iloc[0]["stage 1 permeate flowrate (gpm)"])
            - value(pyunits.convert(m.fs.ro.unit.mixed_permeate[0.0].flow_vol_phase["Liq"], pyunits.gal/pyunits.min)))
    return expr

### Step 6: Solve the parameter estimation problem

Now, we have everything we need for parmest to solve the parameter estimation problem: 

    - ro_parmest
    - data
    - theta_names
    - objective_function


#### Step 6a: Set up the problem

Set up the parameter estimation problem by creating an instance of the parmest 'Estimator' object and feed it the required inputs.

In [ ]:
# create an instance of the parmest estimator
pest = parmest.Estimator(ro_parmest, cleaned_data, theta_names, SSE, tee=False,solver_options={'bound_push':1e-8})
# Attempted Troubleshooting below by setting up intial guesses, but I think this is actually done automatically
# theta_guesses = pd.DataFrame(columns=theta_names)
# print(theta_guesses)
# theta_guesses.loc['fs.ro.unit.A_comp[0,H2O]'] = 7.66e-12
# theta_guesses.loc['fs.ro.unit.B_comp[0,NaCl]'] = 2.22e-8
# print(theta_guesses)

Empty DataFrame
Columns: [fs.ro.unit.A_comp[0,H2O], fs.ro.unit.B_comp[0,NaCl]]
Index: []
                           fs.ro.unit.A_comp[0,H2O]  fs.ro.unit.B_comp[0,NaCl]
fs.ro.unit.A_comp[0,H2O]               7.660000e-12               7.660000e-12
fs.ro.unit.B_comp[0,NaCl]              2.220000e-08               2.220000e-08


In [135]:
pest.objective_at_theta(theta_values=theta_guesses, initialize_parmest_model=True)

Setting RO 1 operating conditions
2026-01-12 12:06:46 [INFO] idaes.init.fs.feed: Initialization Complete.
2026-01-12 12:06:46 [INFO] idaes.init.fs.ro.feed: Initialization Step Complete.
2026-01-12 12:06:46 [INFO] idaes.init.fs.ro.unit.feed_side: Initialization Complete
2026-01-12 12:06:51 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.0,Liq,H2O] has no initial value: setting to 0.0
2026-01-12 12:06:51 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.0,Liq,NaCl] has no initial value: setting to 0.0
2026-01-12 12:06:51 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.1,Liq,H2O] has no initial value: setting to 0.0
2026-01-12 12:06:51 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.1,Liq,NaCl] has no initial value: setting to 0.0
2026-01-12 12:06:51 [INFO] idaes.watertap.core.util.

AssertionError: 

#### Step 6b: Solve the parameter estimation problem 

Solve the parameter estimation problem by calling theta_est. This will use the entire data set to perform the parameter estimation. 

There are additional options for solving and testing. Further details can be found in the [parmest documentation](https://pyomo.readthedocs.io/en/6.7.0/contributed_packages/parmest/index.html).

In [ ]:
# solve the parameter estimation problem
obj, theta = pest.theta_est()

Setting RO 1 operating conditions
2026-01-12 11:53:16 [INFO] idaes.init.fs.feed: Initialization Complete.
2026-01-12 11:53:17 [INFO] idaes.init.fs.ro.feed: Initialization Step Complete.
2026-01-12 11:53:17 [INFO] idaes.init.fs.ro.unit.feed_side: Initialization Complete
2026-01-12 11:53:23 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.0,Liq,H2O] has no initial value: setting to 0.0
2026-01-12 11:53:23 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.0,Liq,NaCl] has no initial value: setting to 0.0
2026-01-12 11:53:23 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.1,Liq,H2O] has no initial value: setting to 0.0
2026-01-12 11:53:23 [INFO] idaes.watertap.core.util.initialization: variable fs.ro.unit.feed_side.material_flow_dx[0.0,0.1,Liq,NaCl] has no initial value: setting to 0.0
2026-01-12 11:53:23 [INFO] idaes.watertap.core.util.

In [ ]:
# display results
print("theta:\n", theta)
print("Objective function value:", obj)

theta:
 fs.ro.unit.A_comp[0.0,H2O]     1.252812e-12
fs.ro.unit.B_comp[0.0,NaCl]    5.593753e-06
dtype: float64
Objective function value: 5.62429600422016


### Step 7: Visualize results

Finally, we can visualize the results using ***matplotlib*** to create a plot of the data and the parameter estimatation fit. 

#### Step 7.1: Define the model with the optimal parameters

In [ ]:
def ro_opt():
    # Build the flowsheet
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    m.fs.properties = props.SeawaterParameterBlock()
    m.fs.feed = Feed(property_package=m.fs.properties)
    m.fs.RO = RO(
        property_package=m.fs.properties,
        has_pressure_change=True,
        concentration_polarization_type=ConcentrationPolarizationType.none,
        mass_transfer_coefficient=MassTransferCoefficient.none,
    )

    m.fs.s00 = Arc(source=m.fs.feed.outlet, destination=m.fs.RO.inlet)
    TransformationFactory("network.expand_arcs").apply_to(m)

    # Set initial conditions
    m.fs.feed.properties[0].flow_vol_phase.fix(gpm_to_m3ps * 8)
    m.fs.feed.properties[0].temperature.fix(273.15 + 25)
    m.fs.feed.properties[0].pressure.fix(psi_to_pascal * 188)
    m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(0.001)

    m.fs.RO.area.fix(28.8)  # Data is for 4 membrane elements
    m.fs.RO.permeate.pressure[0].fix(101325)
    m.fs.RO.deltaP.fix(-psi_to_pascal * 24.6)

    # Set optimal values of parameters from parmest
    m.fs.RO.A_comp[0, "H2O"].fix(theta.iloc[0])
    m.fs.RO.B_comp[0, "TDS"].fix(theta.iloc[1])

    # Set scaling factors
    m.fs.properties.set_default_scaling(
        "flow_mass_phase_comp", 1e1, index=("Liq", "H2O")
    )
    m.fs.properties.set_default_scaling(
        "flow_mass_phase_comp", 1e6, index=("Liq", "TDS")
    )
    iscale.set_scaling_factor(m.fs.RO.area, 1e-1)
    iscale.calculate_scaling_factors(m)

    # Initialize the system
    solver.solve(m.fs.feed)
    propagate_state(m.fs.s00)
    m.fs.RO.initialize(outlvl=idaeslog.ERROR)

    assert degrees_of_freedom(m) == 0

    return m

#### Step 7.2: Create a table to save model results

In [ ]:
# Create a new table to save model results
model_results = full_data.copy()
m = ro_opt()
for i in range(model_results.shape[0]):
    # Conversion factors
    psi_to_pascal = 6894.75  # Pressure conversion
    gpm_to_m3ps = 6.309e-005  # Vol. flow conversion

    # Update operating conditions
    m.fs.feed.properties[0].flow_vol_phase.fix(
        gpm_to_m3ps * float(model_results.iloc[i]["flow_vol_in"])
    )
    m.fs.feed.properties[0].pressure.fix(
        psi_to_pascal * float(model_results.iloc[i]["pressure_in"])
    )
    m.fs.feed.properties[0].mass_frac_phase_comp["Liq", "TDS"].fix(
        float(model_results.iloc[i]["mass_frac_TDS_in"])
    )
    m.fs.RO.deltaP.fix(-psi_to_pascal * float(model_results.iloc[i]["deltaP"]))

    results = solver.solve(m, tee=False)
    assert_optimal_termination(results)

    model_results.iloc[i, 5] = (
        value(m.fs.RO.mixed_permeate[0.0].flow_vol_phase["Liq"]) / gpm_to_m3ps
    )
    model_results.iloc[i, 6] = (
        value(m.fs.RO.mixed_permeate[0.0].mass_frac_phase_comp["Liq", "TDS"]) / 1e-6
    )

NameError: name 'full_data' is not defined

#### Step 7.3: Visualize model vs data comparison

In [ ]:
plt.figure()
plt.scatter(
    full_data.index, full_data["flow_vol_permeate"], label="Data", color="red", s=15
)
plt.plot(model_results.index, model_results["flow_vol_permeate"], label="Model")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure()
plt.scatter(
    full_data.index,
    full_data["mass_frac_TDS_permeate"],
    label="Data",
    color="red",
    s=15,
)
plt.plot(model_results.index, model_results["mass_frac_TDS_permeate"], label="Model")
plt.legend()
plt.grid(True)
plt.show()